# Matching two pandas DataFrames by column content

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import pickle

Load the data:

In [2]:
dfc = pd.read_csv("./data/work_strings.csv",usecols=['VAR_0200','VAR_0237'],low_memory=False)
dfc.head()

,VAR_0200,VAR_0237
0,FT LAUDERDALE,FL
1,SANTEE,CA
2,REEDSVILLE,WV
3,LIBERTY,TX
4,FRANKFORT,IL


In [3]:
dfi = pd.read_csv("./data/work_integers.csv",usecols=['VAR_0241'],low_memory=False)
dfi.head()

,VAR_0241
0,33324.0
1,92071.0
2,26547.0
3,77575.0
4,60423.0


In [4]:
db = pd.read_csv('data/cities_reduced.csv',index_col=0)
db.head()

,city_ascii,state_id,population,density,zips,county_fips
0,New York,NY,18713220,10715.0,11229 11226 11225 11224 11222 11221 11220 1138...,36061
1,Los Angeles,CA,12750807,3276.0,90291 90293 90292 91316 91311 90037 90031 9000...,6037
2,Chicago,IL,8604203,4574.0,60018 60649 60641 60640 60643 60642 60645 6064...,17031
3,Miami,FL,6445545,5019.0,33129 33125 33126 33127 33128 33149 33144 3314...,12086
4,Dallas,TX,5743938,1526.0,75287 75098 75233 75254 75251 75252 75253 7503...,48113


Prepare the four functions:

In [5]:
def f1():
    
    # do the matching
    for i, (c,s,z) in enumerate(df.values):  
        
        if s != s or z != z:
            continue
            
        z = str(int(z))
        
        # first by zip
        db_matched = db[(db['state_id'] == s) & (db['zips'].str.contains(z))]
    
        if not db_matched.empty:

            n[i] = db_matched['population'].values[0]
            d[i] = db_matched['density'].values[0]

        
        if n[i] == 0:
            
            #then by city
            db_matched = db[(db['state_id'] == s) & (db['city_ascii'].str.upper() == c)]
    
            if not db_matched.empty:
            
                n[i] = db_matched['population'].values[0]
                d[i] = db_matched['density'].values[0]
                
            else:
                
                logs.append((i,c,s))


In [6]:
def f2():
    
    # do the matching
    for i, (c,s,z) in enumerate(df.values):  
        
        if s != s or z != z:
            continue
            
        z = str(int(z))
       
        # by zip and city in one go
        db_matched = db[(db['state_id'] == s) & ((db['zips'].str.contains(z))|(db['city_ascii'].str.upper()== c))]
    
        if not db_matched.empty:

            n[i] = db_matched['population'].values[0]
            d[i] = db_matched['density'].values[0]
    
        else:
                
            logs.append((i,c,s))


In [7]:
dbs = dict()

sid = db['state_id'].unique()
print(len(sid),sid) # DC, PR included

for id in sid:
    dbs[id] = db[db['state_id'] == id]
    
print(len(dbs.keys()))

# print the last 3, just 2 entries for each
for check in list(dbs.keys())[-3:]:
    print(dbs[check].head(2))

52 ['NY' 'CA' 'IL' 'FL' 'TX' 'PA' 'GA' 'DC' 'MA' 'AZ' 'WA' 'MI' 'MN' 'CO'
 'MD' 'NV' 'OR' 'MO' 'OH' 'IN' 'NC' 'VA' 'WI' 'RI' 'UT' 'TN' 'LA' 'KY'
 'OK' 'CT' 'NE' 'HI' 'NM' 'AL' 'SC' 'KS' 'IA' 'AR' 'ID' 'PR' 'MS' 'NJ'
 'AK' 'NH' 'ND' 'ME' 'SD' 'WV' 'MT' 'DE' 'VT' 'WY']
52
     city_ascii state_id  population  density  \
379       Dover       DE      117282    622.0   
690  Wilmington       DE       70166   2484.0   

                                                  zips  county_fips  
379                      19902 19901 19904 19905 19906        10001  
690  19809 19802 19801 19806 19805 19850 19880 1988...        10003  
      city_ascii state_id  population  density                     zips  \
416   Burlington       VT      109763   1604.0  05405 05401 05408 05406   
2335       Barre       VT       20421    834.0                    05641   

      county_fips  
416         50007  
2335        50023  
    city_ascii state_id  population  density                           zips  \
589   

In [8]:
def f3():
    
    # prepare subsamples of db, dictionaries with key = state_id 
    dbs = dict()
    
    sid = db['state_id'].unique()
    
    for id in sid:
        
        dbs[id] = db[db['state_id'] == id]   
    
    
    # do the matching
    for i, (c,s,z) in enumerate(df.values):  
        
        if s != s or z != z:
            continue
            
        z = str(int(z))
       
        # first by zip
        db_matched = dbs[s][dbs[s]['zips'].str.contains(z)]
    
        if not db_matched.empty:

            n[i] = db_matched['population'].values[0]
            d[i] = db_matched['density'].values[0]
    
        else:
            
            #then by city
            if n[i] == 0:
                
                db_matched = dbs[s][dbs[s]['city_ascii'].str.upper() == c]
                                 
                if not db_matched.empty:
            
                    n[i] = db_matched['population'].values[0]
                    d[i] = db_matched['density'].values[0]
                
                else:
                
                    logs.append((i,c,s))


In [9]:
def f4():
    
    # prepare subsamples of db, dictionaries with key = state_id 
    dbs = dict()
    
    sid = db['state_id'].unique()
    
    for id in sid:
        
        dbs[id] = db[db['state_id'] == id]   
    
    
    # do the matching
    for i, (c,s,z) in enumerate(df.values):  
        
        if s != s or z != z:
            continue
            
        z = str(int(z))
        
        # by zip and city in one go
        db_matched = dbs[s][(dbs[s]['zips'].str.contains(z)) | (dbs[s]['city_ascii'].str.upper()== c)]
    
        if not db_matched.empty:

            n[i] = db_matched['population'].values[0]
            d[i] = db_matched['density'].values[0]
    
        else:
                
            logs.append((i,c,s))
            

and let's run them:

In [10]:
def check_time(fun):
    
    t1 = dt.now()
    
    global n, d, logs, df
    
    n = [0] * dfc.shape[0]
    d = [0] * dfc.shape[0]
    logs = []
    
    df = pd.concat((dfc,dfi),axis=1)
    
    fun()
    
    t2 = dt.now()

    print(fun.__name__,':',round((t2-t1).total_seconds()/60,1),'min elapsed,',\
    '{} ({}%) matched'.format( sum(np.array(n)>0), round(sum(np.array(n)>0)/len(n)*100),1))   

In [11]:
for f in [f1,f2,f3,f4]:
    
    check_time(f)

f1 : 21.8 min elapsed, 141449 (97.0%) matched
f2 : 36.0 min elapsed, 141449 (97.0%) matched
f3 : 1.9 min elapsed, 141449 (97.0%) matched
f4 : 3.2 min elapsed, 141449 (97.0%) matched


In [12]:
n[:5],d[:5],logs[:5]

([106306, 3220118, 604, 6435, 19448],
 [1175.0, 1686.0, 359.0, 80.0, 807.0],
 [(16, 'SHELBY GAP', 'KY'),
  (48, 'COLUMBIA STATION', 'OH'),
  (129, 'MAUK', 'GA'),
  (161, 'CARTERSVILLE', 'VA'),
  (184, 'CHAPPELLS', 'SC')])

Finally, let's check how long does writing down the files take:

In [13]:
t1 = dt.now()

# save to file
with open('./data/test_pop.pkl', 'wb') as f:
    pickle.dump(n, f)

with open('./data/test_den.pkl', 'wb') as f:
    pickle.dump(d, f)
    
with open('./data/test_logs.pkl', 'wb') as f:
    pickle.dump(logs, f)

t2 = dt.now()
print(round((t2-t1).total_seconds(),1),'secs elapsed')

0.6 secs elapsed
